# Gep top20 recommendation
## Ranking data: knn_tfidf_ranking_v2

In [1]:
import caffeine
caffeine.on(display=False)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /Users/anhtth/Library/CloudStorage/OneDrive-UniversityofTwente/2023 UT- THESIS/1-Code/0.cb12_main/nb_recsys_tabular


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Load ranking data (generated by KNN-TFIDF features)
ranking_data = pd.read_csv('./nb_recsys_tabular/knn_tfidf_ranking_v2.csv')

In [8]:
ranking_data.head()

,UserID,JobID,label,City,State,Country
0,13,824471,0,0.0,0.0,1.0
1,13,7415,0,0.0,0.0,1.0
2,13,759423,0,0.0,0.0,1.0
3,13,729658,0,0.0,0.0,1.0
4,13,976335,0,0.0,0.0,1.0


In [9]:
# Load input: train_data_flat, test_data_flat
train_data_flat = pd.read_csv('./xai_posthoc/train_data_flat.csv')
test_data_flat = pd.read_csv('./xai_posthoc/test_data_flat.csv')

In [10]:
features_flat = pd.read_csv('./nb_recsys_tabular/features_flat_tfidf.csv')

In [11]:
features_flat.head()

,UserID,JobID,City,State,Country,DegreeType,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,...,job_matrix_90,job_matrix_91,job_matrix_92,job_matrix_93,job_matrix_94,job_matrix_95,job_matrix_96,job_matrix_97,job_matrix_98,job_matrix_99
0,7,309823,1.0,1.0,1.0,1,2,13.0,1,0,...,0.0,0.417426,0.000000,0.000000,0.065994,0.070672,0.101036,0.073804,0.0,0.055815
1,7,703889,1.0,1.0,1.0,1,2,13.0,1,0,...,0.0,0.105243,0.000000,0.000000,0.116469,0.000000,0.044579,0.130255,0.0,0.049252
2,7,566574,0.0,0.0,1.0,1,2,13.0,1,0,...,0.0,0.052103,0.077570,0.000000,0.000000,0.000000,0.264839,0.000000,0.0,0.048767
3,7,481216,0.0,0.0,1.0,1,2,13.0,1,0,...,0.0,0.000000,0.125297,0.204315,0.000000,0.000000,0.000000,0.156244,0.0,0.078773
4,9,809208,1.0,1.0,1.0,1,3,3.0,1,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.240853,0.000000,0.0,0.266103


In [12]:
def get_u_idx(u_id):
    user = user_set[user_set.UserID==u_id]
    u_idx = user.index.values[0]
    return u_idx

In [13]:
def get_j_idx(j_id):
    job = job_set[job_set.JobID==j_id]
    j_idx = job.index.values[0]
    return j_idx

In [14]:
train_data_flat.head()

,UserID,JobID,label,City,State,Country,DegreeType,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,...,job_matrix_90,job_matrix_91,job_matrix_92,job_matrix_93,job_matrix_94,job_matrix_95,job_matrix_96,job_matrix_97,job_matrix_98,job_matrix_99
0,7,309823,1,1.0,1.0,1.0,1,2,13.0,1,...,0.0,0.417426,0.000000,0.000000,0.065994,0.070672,0.101036,0.073804,0.0,0.055815
1,7,703889,1,1.0,1.0,1.0,1,2,13.0,1,...,0.0,0.105243,0.000000,0.000000,0.116469,0.000000,0.044579,0.130255,0.0,0.049252
2,7,566574,0,0.0,0.0,1.0,1,2,13.0,1,...,0.0,0.052103,0.077570,0.000000,0.000000,0.000000,0.264839,0.000000,0.0,0.048767
3,7,481216,0,0.0,0.0,1.0,1,2,13.0,1,...,0.0,0.000000,0.125297,0.204315,0.000000,0.000000,0.000000,0.156244,0.0,0.078773
4,9,809208,1,1.0,1.0,1.0,1,3,3.0,1,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.240853,0.000000,0.0,0.266103


In [15]:
train_data_flat.columns

Index(['UserID', 'JobID', 'label', 'City', 'State', 'Country', 'DegreeType',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       ...
       'job_matrix_90', 'job_matrix_91', 'job_matrix_92', 'job_matrix_93',
       'job_matrix_94', 'job_matrix_95', 'job_matrix_96', 'job_matrix_97',
       'job_matrix_98', 'job_matrix_99'],
      dtype='object', length=170)

In [16]:
# Get job_matrix_features
job_matrix_idx = range(100)
features_job_matrix = list(map(lambda x: 'job_matrix_' + str(x), job_matrix_idx))

In [17]:
# Get work_matrix_features
work_matrix_idx = range(50)
features_work_matrix = list(map(lambda x: 'work_matrix_' + str(x), work_matrix_idx))

In [18]:
# Get user profile features
user_profile_features = ["DegreeType", "WorkHistoryCount", "TotalYearsExperience", "CurrentlyEmployed", 
                                                "ManagedOthers", "ManagedHowMany"]

In [19]:
# Get location features
location_features = ['City', 'State', 'Country']

In [20]:
from topN_flat import *

# Start topN for a given user_id

- features_job_matrix: feature names for job matrix
- features_work_matrix: feature names for work history matrix
- user_profile_features: list of features for user profile

In [21]:
# Load pretrained models
import pickle

model_name = './nb_baseline_tabular/cb12_logreg_flat.pikle'
logreg = pickle.load(open(model_name, "rb"))

In [22]:
%%time
rec_result_logreg = topN_flat(model = logreg, 
           N = 20, 
           ranking_data = ranking_data, 
           features_flat = features_flat)

CPU times: user 52min 37s, sys: 6.48 s, total: 52min 43s
Wall time: 52min 47s


In [23]:
# Function: get rec_result corresponding to a user id
# rec_result: overall rec_recsult dictionary (output of topN_flat)
def get_rec_result_df(u_id, rec_result):
    rec_20_df = pd.DataFrame(rec_result[u_id])
    rec_cols = ['JobID', 'Y_prob', 'Y_pred']
    rec_20_df.columns = rec_cols
    rec_20_df['UserID'] = u_id
    rec_20_df['rank'] = rec_20_df.groupby('UserID').cumcount()
    return rec_20_df


In [24]:
user_ids = list(ranking_data.groupby('UserID').UserID.unique().astype('int'))

In [25]:
%%time
final_rec_result = pd.DataFrame(columns = ['JobID', 'Y_prob', 'Y_pred','UserID','rank'])

for u_id in user_ids:
    
    temp_df = get_rec_result_df(u_id=u_id, 
                                rec_result=rec_result_logreg)
    # print(u_id, len(temp_df))
    final_rec_result = pd.concat([final_rec_result, temp_df])

final_rec_result = final_rec_result[['UserID','JobID', 'Y_prob', 'Y_pred', 'rank']]

CPU times: user 12.9 s, sys: 188 ms, total: 13.1 s
Wall time: 13.1 s


In [26]:
final_rec_result.head()

,UserID,JobID,Y_prob,Y_pred,rank
0,13,248446,0.970798,1,0
1,13,367918,0.937227,1,1
2,13,580991,0.445008,0,2
3,13,172155,0.351762,0,3
4,13,799192,0.340405,0,4


In [27]:
final_rec_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74320 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UserID  74320 non-null  object 
 1   JobID   74320 non-null  object 
 2   Y_prob  74320 non-null  float64
 3   Y_pred  74320 non-null  object 
 4   rank    74320 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.4+ MB


In [29]:
final_rec_result.to_csv('./output_topN_tabular/rec_result_logreg_ranking_v2.csv', header=True, index=False)